# Learn RL Notes Part 4

# 9 Exploration and Exploitation
## 9.1 Introduction
** Exploration vs. Exploitation Dilemma **
* Online decision-making involves a fundamental choice:
    * Exploitation Make the best decision given current information
    * Exploration Gather more information
* The best long-term strategy may involve short-term sacrifices
* Gather enough information to make the best overall decisions

** Examples **
* Restaurant Selection
    * Exploitation Go to your favourite restaurant
    * Exploration Try a new restaurant
* Online Banner Advertisements
    * Exploitation Show the most successful advert
    * Exploration Show a different advert
* Oil Drilling
    * Exploitation Drill at the best known location
    * Exploration Drill at a new location
* Game Playing
    * Exploitation Play the move you believe is best
    * Exploration Play an experimental move

** Principles **
* Naive Exploration
    * Add noise to greedy policy (e.g. $\epsilon$-greedy)
* Optimistic Initialisation
    * Assume the best until proven otherwise
* Optimism in the Face of Uncertainty
    * Prefer actions with uncertain values
* Probability Matching
    * Select actions according to probability they are best
* Information State Search
    * Lookahead search incorporating value of information
    
## 9.2 Multi-Armed Bandits
** The Multi-Armed Bandit **
* A multi-armed bandit is a tuple $<A, R>$
* A is a known set of m actions (or “arms”)
* $R^a (r ) = P [r |a]$ is an unknown probability distribution over rewards
* At each step t the agent selects an action $a_t ∈ A$
* The environment generates a reward $r_t ∼ R^{a_t}$
* The goal is to maximise cumulative reward $\sum^t_{\tau = 1}r_{\tau}$

### 9.2.1 Regret
** Regret **
* The action-value is the mean reward for action a, $$Q(a) = \mathbb{E} [r |a]$$
* The optimal value $V^∗$ is $$V^∗ = Q(a^∗ ) = \underset{a \in A}{max} Q(a) $$
* The regret is the opportunity loss for one step $$l_t = E [V^∗ − Q(a_t )]$$
* The total regret is the total opportunity loss 
$$L_t = \mathbb{E} \left[ \sum_{\tau = 1}^{t} V^* - Q(a_{\tau}) \right]$$
* Maximise cumulative reward ≡ minimise total regret

** Counting regret** 
<img width=600 src="images/rl-xx-counting-regrets.png" />

** Linear and Sub-linear Regret**
<img width=600 src="images/rl-xx-sublinear-regret.png" />

### 9.2.2 Greedy and $\epsilon$-greedy algorithms

** Greedy Algorithm **
* We consider algorithms that estimate $\hat{Q}_t (a) ≈ Q(a)$
* Estimate the value of each action by Monte-Carlo evaluation
$$ \hat{Q}_t (a) = \frac{1}{N_t(a)}\sum^T_{t=1}r_t \mathbf{1}(a_t =a)$$
* The greedy algorithm selects action with highest value
$$a_t^∗ = \underset{a \in A}{argmax} \hat{Q}_t(a)$$
* Greedy can lock onto a suboptimal action forever
* ⇒ Greedy has linear total regret

** $\epsilon$-Greedy Algorithm **
* The $\epsilon$-greedy algorithm continues to explore forever
    * With probability 1 − $\epsilon$ select $a = \underset{a \in A}{argmax} \hat{Q}(a)$
    * With probability $\epsilon$ select a random action
* Constant $\epsilon$ ensures minimum regret
$$l_t ≥ \frac{\epsilon}{A}\sum_{a \in A}\Delta a$$
* ⇒ -greedy has linear total regret

** Optimistic Initialisation **
* Simple and practical idea: initialise Q(a) to high value
* Update action value by incremental Monte-Carlo evaluation
* Starting with N(a) > 0
$$ \hat{Q}_t (a_t ) = \hat{Q}_{t-1} + \frac{1}{N_t(a_t)}(r(t) -\hat{Q}_{t-1)}$$
* Encourages systematic exploration early on
* But can still lock onto suboptimal action
* ⇒ greedy + optimistic initialisation has linear total regret
* ⇒ $\epsilon$-greedy + optimistic initialisation has linear total regret

** Decaying $\epsilon_t$-Greedy Algorithm **
* Pick a decay schedule for $\epsilon_1 , \epsilon_2 $, ...
* Consider the following schedule
$$ c > 0 \\
d = \underset{a \mid \Delta a > 0}{min} \Delta i \\
\epsilon_t  = min \left \{ 1, \frac{c|A|}{d^2 t} \right \} $$
* has logarithmic asymptotic total regret!
* Unfortunately, schedule requires advance knowledge of gaps
* Goal: find an algorithm with sublinear regret for any multi-armed bandit (without knowledge of R)

### 9.2.3 Upper Confidence Bound
** Lower Bound **
* The performance of any algorithm is determined by similarity between optimal arm and other arms
* Hard problems have similar-looking arms with different means
* This is described formally by the gap $∆_a$ and the similarity in distributions $KL(R^a ||R^{a_∗})$

** Theorem (Lai and Robbins) **

Asymptotic total regret is at least logarithmic in number of steps
$$\underset{t \to \infty}{lim} L_t ≥ log t \sum_{a|∆_a >0} \frac{∆_a}{KL(R^a ||R^{a^∗})} $$

<img width=600 src="images/rl-xx-optimism-uncertainty.png" />
<img width=600 src="images/rl-xx-optimism-uncertainty2.png" />

**Upper Confidence Bound**

* Estimate an upper confidence $\hat{U}_t (a)$ for each action value
* Such that $Q(a) \leq \hat{Q}_t(a)+ \hat{U}_t (a)$ with high probability
* This depends on the number of times N(a) has been selected
    * Small $N_t (a) \implies large \hat{U}_t(a)$ (estimated value is uncertain)
    * Large $N_t (a) \implies small \hat{U}_t(a)$ (estimated value is accurate)
* Select action maximising **Upper Confidence Bound (UCB)**
$$ a_t = \underset{a \in A}{argmax}\hat{Q}_t (a) + \hat{U}_t (a) $$

** Hoeffding’s Inequality **
Theorem (Hoeffding’s Inequality)
Let $X_1 , ..., X_t$ be i.i.d. random variables in [0,1], and let $\overline{X}_t = \frac{1}{t}\sum_{\tau=1}^t X_{\tau}$ be the **sample mean**. Then
$$\mathbb{P} [ \mathbb{E} [X] > \overline{X}_t + u] ≤ e^{−2tu^2}$$

* We will apply Hoeffding’s Inequality to rewards of the bandit
* conditioned on selecting action a
$$\mathbb{P} [Q(a) > \hat{Q}_t (a) + U_t (a)] \leq e^{ −2N_t(a)U_t(a)^2}$$

** Calculating Upper Confidence Bounds **
* Pick a probability p that true value exceeds UCB
* Now solve for $U_t (a)$
$$ e^{−2N_t(a)U_t(a)^2} = p \\
U_t (a) = \sqrt {\frac {− log p} {2N_t (a)}}$$
* Reduce p as we observe more rewards, e.g. $p = t^{−4}$
* Ensures we select optimal action as $t \to \infty$ 
$$ U_t (a) = \sqrt {\frac {2 log t}{N_t (a)}} $$

**UCB1**

This leads to the UCB1 algorithm
$$a_t = \underset{a \in A}{argmax} Q(a) + \sqrt{\frac{2 log t}{N_t (a)}} $$

**Theorem** The UCB algorithm achieves logarithmic asymptotic total regret
$$ \underset{t \to \infty}{lim} L_t \leq 8 log t \sum_{a \mid \Delta a > 0}\Delta a$$

### 9.2.4 Bayesian Bandits
** Bayesian Bandits **
* So far we have made no assumptions about the reward distribution R
    * Except bounds on rewards
* Bayesian bandits exploit prior knowledge of rewards, p[R]
* They compute posterior distribution of rewards $p [R | h_t ]$
    * where $h_t = a_1 , r_1 , ..., a_{t−1} , r_{t−1}$ is the history
* Use posterior to guide exploration
    * Upper confidence bounds (Bayesian UCB)
    * Probability matching (Thompson sampling)
* Better performance if prior knowledge is accurate

<img width=600 src="images/rl-xx-bayesian-independent-gaussian.png" />

** Probability Matching **
* Probability matching selects action a according to probability that a is the optimal action
$$ π(a | h_t ) = \mathbb{P} [Q(a) > \hat{Q}(a ), \forall a' \neq a \mid h_t ]$$
* Probability matching is optimistic in the face of uncertainty
    * Uncertain actions have higher probability of being max
* Can be difficult to compute analytically from posterior

** Thompson Sampling **
* Thompson sampling implements probability matching
$$ \begin{align*} \\
\pi(a | h_t ) &= \mathbb{P} [Q(a) > Q(a' ), \forall a' \neq a | h_t] \\
&= \mathbb{E}_{R|h_t} \mathbf{1}(a = \underset{a \in A}{argmax}) Q(a) \\
\end{align*}$$

* Use Bayes law to compute posterior distribution $p [R | h_t ]$
* Sample a reward distribution R from posterior
* Compute action-value function $Q(a) = \mathbb{E} [R_a ]$
* Select action maximising value on sample, $a_t = \underset{a \in A}{argmax} Q(a)$
* **Thompson sampling achieves Lai and Robbins lower bound!**

### 9.2.5 Information State Search
** Value of Information **
* Exploration is useful because it gains information
* Can we quantify the value of information?
    * How much reward a decision-maker would be prepared to pay in order to have that information, prior to making a decision
    * Long-term reward after getting information - immediate reward
* Information gain is higher in uncertain situations
* Therefore it makes sense to explore uncertain situations more
* **If we know value of information, we can trade-off exploration and exploitation optimally**

<img width=600 src="images/rl-xx-information-space.png" />

<img width=600 src="images/rl-xx-bernoulli-bandit.png" />

** Solving Information State Space Bandits **
* We now have an infinite MDP over information states
* This MDP can be solved by reinforcement learning
* Model-free reinforcement learning
    * e.g. Q-learning (Duff, 1994)
* Bayesian model-based reinforcement learning
    * e.g. Gittins indices (Gittins, 1979)
    * This approach is known as Bayes-adaptive RL
    * Finds Bayes-optimal exploration/exploitation trade-off with respect to prior distribution

<img width=600 src="images/rl-xx-bayes-adaptive-bernoulli-bandits.png" />
<img width=600 src="images/rl-xx-bayes-mdp.png" />

** Gittins Indices for Bernoulli Bandits **
* Bayes-adaptive MDP can be solved by dynamic programming
* The solution is known as the **Gittins index**
* Exact solution to Bayes-adaptive MDP is typically intractable
    * Information state space is too large
* Recent idea: apply simulation-based search (Guez et al. 2012)
    * Forward search in information state space
    * Using simulations from current information state

## 9.3 Contextual Bandits
<img width=600 src="images/rl-xx-contextual-bandits.png" />
<img width=600 src="images/rl-xx-linear-regression.png" />
<img width=600 src="images/rl-xx-linear-ucb.png" />
<img width=600 src="images/rl-xx-linear-ucb-intuition.png" />
<img width=600 src="images/rl-xx-linear-ucb-calculation.png" />

## 9.4 MDPs
** Exploration/Exploitation Principles to MDPs **
* The same principles for exploration/exploitation apply to MDPs
* Naive Exploration
* Optimistic Initialisation
* Optimism in the Face of Uncertainty
* Probability Matching
* Information State Search

### 9.4.1 Optimistic Initialisation: Model-Free RL
* Initialise action-value function Q(s, a) to $\frac {r_{max}}{1-\gamma}$
* Run favourite model-free RL algorithm
    * Monte-Carlo control
    * Sarsa
    * Q-learning
    * .
* courages systematic exploration of states and actions

### 9.4.2 Optimistic Initialisation: Model-Based RL
* Construct an **optimistic** model of the MDP
* Initialise transitions to **go to heaven**
    * (i.e. transition to terminal state with $r_{max}$ reward)
* Solve optimistic MDP by favourite planning algorithm
    * policy iteration
    * value iteration
    * tree search
    * ...
* Encourages systematic exploration of states and actions
    * e.g. RMax algorithm (Brafman and Tennenholtz)
    
### 9.4.3 Upper Confidence Bounds: Model-Free RL
* Maximise UCB on action-value function $Q^π (s, a)$
$$ a_t = \underset{a \in A}{argmax} Q(s_t , a) + U(s_t , a)$$
    * Estimate uncertainty in policy evaluation (easy)
    * Ignores uncertainty from policy improvement
* Maximise UCB on optimal action-value function $Q^∗ (s, a)$
$$ a_t = \underset{a \in A}{argmax} Q(s _t , a) + U_1 (s_t , a) + U_2 (s_t , a)$$
    * Estimate uncertainty in policy evaluation (easy)
    * plus uncertainty from policy improvement (hard)

### 9.4.4 Bayesian Model-Based RL
* Maintain posterior distribution over MDP models
* Estimate both transitions and rewards, $p [\mathcal{P}, \mathcal{R} | h_t ]$
    * where $h_t = s_1 , a_1 , r_2 , ..., s_t$ is the history
* Use posterior to guide exploration
* Upper confidence bounds (Bayesian UCB)
* Probability matching (Thompson sampling)

### 9.4.5 Thompson Sampling: Model-Based RL
* Thompson sampling implements probability matching
$$\begin{align*} \\
π(s, a | h_t ) &= \mathbb{P} [Q^∗ (s, a) > Q^∗ (s, a' ), \forall a \neq a' | h_t] \\
&= \mathbb{E}_{\mathcal{P,R}|h_t} \left[\mathbf{1}(a = \underset{a \in A}{argmax} Q^∗ (s, a))\right]
\end{align*}$$

* Use Bayes law to compute posterior distribution $p [\mathcal{P}, \mathcal{R} | h_t ]$
* **Sample** an MDP P, R from posterior
* Solve MDP using favourite planning algorithm to get $Q^∗ (s, a)$
* Select optimal action for sample MDP, $a_t = \underset{a \in A}{argmax} Q^∗ (s_t , a)$

### 9.4.6 Information State Search in MDPs
* MDPs can be augmented to include information state
* Now the augmented state is $\langle s,\tilde{s} \rangle$
    * where s is original state within MDP
    * and $\tilde{s}$ is a statistic of the history (accumulated information)
* Each action a causes a transition
    * to a new state s' with probability $\mathcal{P}^a_{s, s'}$
    * to a new information state $\tilde{s}'$
* Defines MDP $\tilde{\mathcal{M}}$ in augmented information state space
    $$ \tilde{\mathcal{M}} = \langle \tilde{S},A,\tilde{P},R,\gamma \rangle$$

### 9.4.7 Bayes Adaptive MDPs
* Posterior distribution over MDP model is an information state
$$ \tilde{s}_t = \mathbb{P} [\mathcal{P}, \mathcal{R} \mid h_t]$$
* Augmented MDP over $\langle s, \tilde{s} \rangle$ is called Bayes-adaptive MDP
* Solve this MDP to find optimal exploration/exploitation trade-off (with respect to prior)
* However, Bayes-adaptive MDP is typically enormous
* Simulation-based search has proven effective (Guez et al.)

**Conclusion**
* Have covered several principles for exploration/exploitation
* Naive methods such as $\epsilon$-greedy
* Optimistic initialisation
* Upper confidence bounds
* Probability matching
* Information state search
* Each principle was developed in bandit setting
* But same principles also apply to MDP setting


# 10 Case study - RL in games
## 10.1 State of the Art
** Why Study Classic Games? **
* Simple rules, deep concepts
* Studied for hundreds or thousands of years
* Meaningful IQ test
* Drosophila of artificial intelligence
* Microcosms encapsulating real world issues
* Games are fun!
<img width=600 src="images/rl-game-ai-in-games.png" />
<img width=600 src="images/rl-game-ai-in-games2.png" />

## 10.2 Game Theory
** Optimality in Games **
* What is the optimal policy $π^i$ for ith player?
* If all other players fix their policies $π^{−i}$
* **Best response** $π^i_∗ (π^{−i} )$ is optimal policy against those policies
* **Nash equilibrium** is a joint policy for all players
$$π^i = π_∗^i (π^{−i} )$$
* such that every player’s policy is a best response
* i.e. no player would choose to deviate from Nash

** Single-Agent and Self-Play Reinforcement Learning **
* **Best response** is **solution** to single-agent RL problem
    * Other players become part of the environment
    * Game is reduced to an MDP
    * Best response is optimal policy for this MDP
* **Nash equilibrium** is **fixed-point** of self-play RL
    * Experience is generated by playing games between agents
    $$a_1 ∼ π^1 , a_2 ∼ π^2 , ...$$
    * Each agent learns best response to other players
    * One player’s policy determines another player’s environment
    * All players are adapting to each other

** Two-Player Zero-Sum Games **
* We will focus on a special class of games:
    * A two-player game has two (alternating) players
        * We will name player 1 white and player 2 black
    * A zero sum game has equal and opposite rewards for black and white
    $$R^1 + R^2 = 0$$
* We consider methods for finding Nash equilibria in these games
    * Game tree search (i.e. planning)
    * Self-play reinforcement learning

** Perfect and Imperfect Information Games **
* A perfect information or Markov game is fully observed
    * Chess
    * Checkers
    * Othello
    * Backgammon
    * Go
* An imperfect information game is partially observed
    * Scrabble
    * Poker
* We focus first on perfect information games

## 10.3 Minimax Search

** Minimax **
* A value function defines the expected total reward given joint policies $π = \langle π^1 , π^2 \rangle$
$$v_π (s) = \mathbb{E}_π [G_t \mid S_t = s]$$
* A minimax value function maximizes white’s expected return while minimizing black’s expected return
$$v_∗ (s) = \underset{\pi ^ 1}{max} \underset{\pi ^ 2}{min} v_π (s)$$
* A minimax policy is a joint policy $π = \langle π^1 , π^2 \rangle$ that achieves the minimax values
* There is a unique minimax value function
* A minimax policy is a Nash equilibrium

<img width=600 src="images/rl-game-minimax-search.png" />
<img width=600 src="images/rl-game-minimax-search2.png" />
<img width=600 src="images/rl-game-minimax-search3.png" />
<img width=600 src="images/rl-game-minimax-search4.png" />
<img width=600 src="images/rl-game-minimax-search5.png" />

** Value Function in Minimax Search **
* Search tree grows exponentially
* Impractical to search to the end of the game
* Instead use value function approximator $v(s, w) ≈ v_∗ (s)$
    * aka **evaluation function**, **heuristic function**
* Use value function to estimate minimax value at leaf nodes
* Minimax search run to fixed depth with respect to leaf values

<img width=600 src="images/rl-game-binary-linear-value-function.png" />
<img width=600 src="images/rl-game-deep-blue.png" />
<img width=600 src="images/rl-game-chinook.png" />

## 10.4 Self-Play Reinforcement Learning
<img width=600 src="images/rl-game-self-play-td-learning.png" />
<img width=600 src="images/rl-game-policy-improvement-with-afterstates.png" />
<img width=600 src="images/rl-game-logistello.png" />
<img width=600 src="images/rl-game-reinforcement-learning-in-logistello.png" />
<img width=600 src="images/rl-game-td-gammon-non-linear.png" />
<img width=600 src="images/rl-game-self-play-td-in-td-gammon.png" />
<img width=600 src="images/rl-game-td-gammon-results.png" />

## 10.5 Combining Reinforcement Learning and Minimax Search
<img width=600 src="images/rl-game-simple-td.png" />
<img width=600 src="images/rl-game-simple-td-results.png" />
<img width=600 src="images/rl-game-td-root.png" />
<img width=600 src="images/rl-game-td-root-in-checker.png" />
<img width=600 src="images/rl-game-td-leaf.png" />
<img width=600 src="images/rl-game-td-leaf-in-chess.png" />
<img width=600 src="images/rl-game-td-leaf-in-checker.png" />
<img width=600 src="images/rl-game-treestrap.png" />
<img width=600 src="images/rl-game-treestrap-in-chess.png" />
<img width=600 src="images/rl-game-simulation-based-search.png" />
<img width=600 src="images/rl-game-performance-of-mcts.png" />
<img width=600 src="images/rl-game-simple-mcts-in-maven.png" />

## 10.6 Reinforcement Learning in Imperfect-Information Games
<img width=600 src="images/rl-game-imperfect-info.png" />
<img width=600 src="images/rl-game-imperfect-info-solution.png" />
<img width=600 src="images/rl-game-smooth-uct-search.png" />

## 10.7 Summary
<img width=600 src="images/rl-game-summary.png" />
